In [ ]:
import os
import re
from lxml import etree

DATA_PATH = "/content/drive/My Drive/#7 Fall 2019 Senior/CSC Honor Research/Benjamin Liu/Pediatrics/All xml files"
os.chdir(DATA_PATH)
#!ls -al

namespaces={"v3":"urn:hl7-org:v3",}

class SPLDocument:
    def __init__(self):
        self.myDict = {}
    
    def getMetaData(self,tree,fileName):
        self.myDict['SPLFileName']= fileName
        # Get the drug name
        drugName = tree.xpath(".//v3:genericMedicine/v3:name/text()",namespaces=namespaces)
        
        if(len(drugName)==0):
          return -1

        self.myDict['genericName']= drugName[0]
        # Get the Indication and Usage section text
        indi = []
        indicationText = ""
        query = "//v3:code[@code='34067-9']"
        indication = tree.xpath(query,namespaces=namespaces)
        if len(indication) != 0:
            indication = indication[0]
            for ancestor in indication.iterancestors('{urn:hl7-org:v3}section'):
                elementText = " "
                elementText += "".join(ancestor.itertext())	
                indi.append(elementText)
                break
            indicationText = ''.join(indi)
            self.myDict['indication']= indicationText
        else:
            query = "//v3:code[@code='42229-5']"
            indication = tree.xpath(query,namespaces=namespaces)
            if len(indication)!=0:
                for i in indication:
                    if(i.getnext()!=None and i.getnext().text!=None):
                        if "indication" in i.getnext().text.lower():
                            for ancestor in i.iterancestors('{urn:hl7-org:v3}section'):
                                for des in ancestor.iterdescendants():
                                    if des.tag == '{urn:hl7-org:v3}title' or des.text == None:
                                        continue	
                                    indi.append(des.text)
                            indicationText = ''.join(indi)
                            #print("Indication text == ",indicationText)
                            #self.myDict['indication']= indicationText
                            break
                self.myDict['indication']= indicationText
            else:
                self.myDict['indication']=""
        if self.myDict['indication'] == "":
            print('File without indication: ' + fileName)
            return -1       # Skip this file if indication section NOT PRESENT

        
        # Get the pediatric Use section text
        pedia = []
        elementText = " "

        notPedia = 0
        pattern = -1
        query = "//v3:code[@code='34081-0']"
        pediatricUse = tree.xpath(query,namespaces=namespaces)
        if len(pediatricUse) != 0:
            pediatricUse = pediatricUse[0]
            for ancestor in pediatricUse.iterancestors('{urn:hl7-org:v3}section'):
                for e in ancestor.itertext():
                    elementText += e.strip() 
                pedia.append(elementText)
                break	
            pediaText = ''.join(pedia)
            self.myDict['pediatricUse']= pediaText + '\n'
            pattern = 1
            global count1
            count1 += 1
        else:
            pediatricUse = tree.xpath("//v3:title[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'pediatric use')]",namespaces=namespaces)  #or contains(.,'pediatric use') or contains(.,'PEDIATRIC')]",namespaces=namespaces)
            if len(pediatricUse) != 0:
                pediatricUse = pediatricUse[0]
                for ancestor in pediatricUse.iterancestors('{urn:hl7-org:v3}section'):
                    for e in ancestor.itertext():
                        elementText += e.strip()
                    pedia.append(elementText)
                    break
                pediaText = ''.join(pedia)
                self.myDict['pediatricUse']= pediaText
                pattern = 2
                global count2
                count2 += 1
                #return 2   # to count the number of files
            else:
                pediaNext = []
                pediatricUse = tree.xpath("//v3:linkHtml",namespaces=namespaces) 
                if len(pediatricUse) != 0:
                    for i in pediatricUse:
                        if(i.tail!=None):
                            if i.tail.lower()=="pediatric use" or i.tail.lower()=="pediatric use section":
                                for para in i.itersiblings(preceding=False):
                                    if(para.tag=="{urn:hl7-org:v3}linkHtml"):                                        
                                        break
                                    pediaNext.append(("".join(para.itertext(with_tail=True))).strip())                                
                                break   #turn on after counting files
                    # IF no Pediatric linkHTML exists, set to " "
                    
                    self.myDict['pediatricUse']= "".join(pediaNext)
                    if(len(pediaNext)==0):
                        noPediaPatternFiles.append(fileName)
                        notPedia = 1
                    else:
                        pattern = 3
                        global count3
                        count3 += 1
                else:
                    self.myDict['pediatricUse'] = ""
                    noPediaPatternFiles.append(fileName)
                    notPedia = 1
        
        retVal =  0

        if notPedia==1:
            return 0
        else:
            retVal = pattern
            if (retVal == 1):
                os.makedirs('/content/drive/My Drive/#7 Fall 2019 Senior/CSC Honor Research/Benjamin Liu/Pediatrics/output/train/pedia', exist_ok = True)
                outputfile = "/content/drive/My Drive/#7 Fall 2019 Senior/CSC Honor Research/Benjamin Liu/Pediatrics/output/train/pedia/" + doc.replace('.xml','.txt')
            else:
                os.makedirs('/content/drive/My Drive/#7 Fall 2019 Senior/CSC Honor Research/Benjamin Liu/Pediatrics/output/test/test1/pedia', exist_ok = True)
                outputfile = "/content/drive/My Drive/#7 Fall 2019 Senior/CSC Honor Research/Benjamin Liu/Pediatrics/output/test/test1/pedia/" + doc.replace('.xml','.txt')
            f= open(outputfile,"wb+")
            self.myDict['pediatricUse'] = self.myDict['pediatricUse'].strip().replace('  ','')
            f.write(self.myDict['pediatricUse'].encode('ascii', 'ignore'))
            f.close()

        return retVal

    def getAllData(self, tree, fileName):
        # Get the all text
        all_text = ""
        pedia_text = ""
        elementText = " "

        query = "//v3:structuredBody"
        structBody = tree.xpath(query,namespaces=namespaces)
        if len(structBody) != 0:
            structBody = structBody[0]
            for e in structBody.itertext():
                if re.search('pediatric', e):
                    pedia_text = pedia_text + e.strip()
                else:
                    all_text = all_text + e.strip()

        if(pedia_text != ""):
            pattern = 3
            global count3
            count3 += 1
            os.makedirs('/content/drive/My Drive/#7 Fall 2019 Senior/CSC Honor Research/Benjamin Liu/Pediatrics/output/test/test2/pedia', exist_ok = True)
            outputfile = "/content/drive/My Drive/#7 Fall 2019 Senior/CSC Honor Research/Benjamin Liu/Pediatrics/output/test/test2/pedia/" + doc.replace('.xml','.txt')
            f= open(outputfile,"wb+")
            pedia_text = pedia_text.replace('  ','')
            f.write(pedia_text.encode('ascii', 'ignore'))
            f.close()
        else:
            os.makedirs('/content/drive/My Drive/#7 Fall 2019 Senior/CSC Honor Research/Benjamin Liu/Pediatrics/output/train/nonpedia', exist_ok = True)
            outputfile = "/content/drive/My Drive/#7 Fall 2019 Senior/CSC Honor Research/Benjamin Liu/Pediatrics/output/train/nonpedia/" + doc.replace('.xml','.txt')
            f= open(outputfile,"wb+")
            all_text = all_text.replace('  ','')
            f.write(all_text.encode('ascii', 'ignore'))
            f.close()
            return 0

        return pattern
    
if __name__ == '__main__':
    
    # ## Parsing SPL documents 
    global count1
    global count2
    global count3

    count1 = 0
    count2 = 0
    count3 = 0

    documents = []

    filePath = r"/content/drive/My Drive/#7 Fall 2019 Senior/CSC Honor Research/Benjamin Liu/Pediatrics/All xml files/" #filePath = r"../data/"
    p = etree.XMLParser(huge_tree=True)

    noPediaPatternFiles = []

    for doc in os.listdir(filePath):
        spl = SPLDocument()
        tree = etree.parse(filePath+doc,parser=p)
        return_value = spl.getMetaData(tree, doc)
        if(return_value == 0):
            return_value = spl.getAllData(tree, doc)
    
    print("pattern 1",  count1)
    
    print("pattern 2",  count2)
    
    print("pattern 3",  count3)


In [ ]:
# check how many files are in "All xml files" folder
import os

Dest_path = "/content/drive/My Drive/#7 Fall 2019 Senior/CSC Honor Research/Benjamin Liu/Pediatrics/output/test/test1/pedia/"

count = 0
for doc_num, doc in enumerate(os.listdir(Dest_path)):
  count += 1
print("There are " + str(count) + " files in folder")

In [ ]:
# total 2269 test1, 29000 train
import os
import shutil

Src_path = "/content/drive/My Drive/#7 Fall 2019 Senior/CSC Honor Research/Pediatrics/output/test/test1/pedia/"
Dest_path = "/content/drive/My Drive/#7 Fall 2019 Senior/CSC Honor Research/Benjamin Liu/Pediatrics/output/test/test1/pedia/"

num_copied = 0
for doc_num, doc in enumerate(os.listdir(Src_path)):
  shutil.copy(Src_path + doc, Dest_path + doc)
  num_copied += 1
  print(str(num_copied) + " copied")

In [18]:
# Increase Test1 and Test2
import os
import shutil

# !CHANGE!
Src_path = "/content/drive/My Drive/#7 Fall 2019 Senior/CSC Honor Research/Benjamin Liu/Pediatrics/output/train/pedia/"
Dest_path_test1 = "/content/drive/My Drive/#7 Fall 2019 Senior/CSC Honor Research/Benjamin Liu/Pediatrics/output/test/test1/pedia/"
# Dest_path_test2 : NONE

# random
import random
random.seed(2020)
rand_list = random.sample(range(len(os.listdir(Src_path))), 231) # !CHANGE!

num_moved = 0
for doc_num, doc in enumerate(os.listdir(Src_path)):
  if doc_num in rand_list:
    shutil.copy(Src_path + doc, "/content/drive/My Drive/#7 Fall 2019 Senior/CSC Honor Research/Benjamin Liu/Pediatrics/output/test/test1/temp/" + doc)
    shutil.move(Src_path + doc, Dest_path_test1 + doc)
    num_moved += 1
    if num_moved % 100 == 0:
      print(str(num_moved) + " moved")


# Check how many files in Src_path
count = 0
for doc_num, doc in enumerate(os.listdir(Src_path)):
  count += 1
print("There are " + str(count) + " files in folder")
# Check how many files in Dest_path
count = 0
for doc_num, doc in enumerate(os.listdir(Dest_path_test1)):
  count += 1
print("There are " + str(count) + " files in folder")

100 moved
200 moved
There are 28769 files in folder
There are 2600 files in folder
